In [ ]:
!pip install syft
!pip install fashion

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
# from fashion import fashion

In [0]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [0]:
trainset = dataset.FashionMNIST('./fmnist.',download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [0]:
testset = dataset.FashionMNIST('./fmnist.',download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [8]:
import syft as sy
hook = sy.TorchHook(torch)
aviskar = sy.VirtualWorker(hook, id='aviskar')
bipin = sy.VirtualWorker(hook, id='bipin')

W0626 07:37:42.884648 140436642215808 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0-rc1.so'
W0626 07:37:42.900305 140436642215808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
class Hyperparam():
  def __init__(self):
    self.epochs = 10
    self.batch_size = 64
    self.lr = 0.01
    self.seed = 20
    self.save_model = False
    self.test_batch_size = 100
    self.momentum = 0.5
    self.log_interval = 30

args = Hyperparam()
device = "cpu"

In [0]:
use_cuda = False
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
federated_train_loader = sy.FederatedDataLoader(
     trainset.federate((aviskar,bipin)), batch_size=args.batch_size, shuffle = True, **kwargs 
)

In [0]:

class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model.send(data.location)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get()
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [27]:
%%time
model = FashionNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, testloader)

if (args.save_model):
    torch.save(model.state_dict(), "Fmnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.297894
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.270388
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.256516
Train Epoch: 1 [5760/60032 (10%)]	Loss: 2.218875
Train Epoch: 1 [7680/60032 (13%)]	Loss: 2.107992
Train Epoch: 1 [9600/60032 (16%)]	Loss: 2.018991
Train Epoch: 1 [11520/60032 (19%)]	Loss: 1.680741
Train Epoch: 1 [13440/60032 (22%)]	Loss: 1.539548
Train Epoch: 1 [15360/60032 (26%)]	Loss: 1.253159
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.972984
Train Epoch: 1 [19200/60032 (32%)]	Loss: 1.001128
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.925889
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.879947
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.702568
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.791418
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.777112
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.773149
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.922166
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.706456
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.693013
Train Epoch: